In [2]:
import pandas as pd

In [3]:

import numpy as np
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [4]:
import tensorflow as tf

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from statsmodels.tsa.arima.model import ARIMA

## Step 1: Generate Synthetic Time-Series Financial Dataset

In [14]:
np.random.seed(42)
data_size = 5000
date_range = pd.date_range(start="2010-01-01", periods=data_size, freq="D")

df = pd.DataFrame({
    "Date": date_range,
    "Customer_Age": np.random.randint(18, 70, data_size),
    "Annual_Income": np.random.randint(20000, 180000, data_size),
    "Spending_Score": np.random.randint(1, 100, data_size),
    "Credit_Score": np.random.randint(300, 950, data_size),
    "Loan_Amount": np.random.randint(1000, 60000, data_size),
    "Default": np.random.choice(["Yes", "No"], data_size, p=[0.2, 0.8]),  # Categorical Target
    "Stock_Price": np.cumsum(np.random.randn(data_size) * 2 + 0.5) + 100  # Simulated Stock Market Prices
})


## Step 2: Data Cleaning & Preprocessing

In [15]:

df.drop_duplicates(inplace=True)
df.fillna(df.select_dtypes(include=["number"]).median(), inplace=True)

# Convert categorical target variable to numeric
encoder = LabelEncoder()
df["Default_Label"] = encoder.fit_transform(df["Default"])  # Yes = 1, No = 0

# Normalize numerical features
scaler = StandardScaler()
numerical_cols = ["Customer_Age", "Annual_Income", "Spending_Score", "Credit_Score", "Loan_Amount", "Stock_Price"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


## Step 3: Train-Test Split for Machine Learning

In [16]:
X = df[numerical_cols]
y = df["Default_Label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 4: Train a Neural Network Model for Loan Default Prediction

In [17]:
model = Sequential([
    Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)


C:\Users\Dom3n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



## Step 5: Time-Series Forecasting Using ARIMA

In [18]:
arima_model = ARIMA(df["Stock_Price"], order=(5,1,0))
fitted_arima = arima_model.fit()
forecast_arima = fitted_arima.forecast(steps=30)  # 30-day forecast

forecast_dates = pd.date_range(start=df["Date"].iloc[-1], periods=31, freq="D")[1:]
forecast_df = pd.DataFrame({"Date": forecast_dates, "Forecasted_Price": forecast_arima})

## Step 6: Time-Series Forecasting Using LSTM

In [19]:
lstm_train = df["Stock_Price"].values.reshape(-1, 1)
scaler_lstm = StandardScaler()
lstm_train_scaled = scaler_lstm.fit_transform(lstm_train)

X_lstm, y_lstm = [], []
for i in range(30, len(lstm_train_scaled)):
    X_lstm.append(lstm_train_scaled[i-30:i, 0])
    y_lstm.append(lstm_train_scaled[i, 0])

X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

lstm_model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(X_lstm.shape[1], 1)),
    LSTM(50, activation="relu"),
    Dense(1)
])

lstm_model.compile(optimizer="adam", loss="mse")
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=32, verbose=0)

future_lstm = lstm_model.predict(X_test_lstm[:30])  # LSTM Forecast
future_lstm = scaler_lstm.inverse_transform(future_lstm)

forecast_df["LSTM_Forecast"] = future_lstm[:30]

C:\Users\Dom3n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step


## Step 7: Create Interactive Dashboard with Dash

In [20]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Financial Data Analysis & Forecasting"),
    html.H3("Neural Network Loan Default Prediction Accuracy: {:.2f}".format(model.evaluate(X_test, y_test, verbose=0)[1])),

    dcc.Dropdown(
        id="feature",
        options=[{"label": col, "value": col} for col in numerical_cols],
        value="Annual_Income"
    ),

    dcc.Graph(id="feature-vs-default"),
    dcc.Graph(id="stock-forecast")
])

@app.callback(
    Output("feature-vs-default", "figure"),
    Input("feature", "value")
)
def update_graph(feature):
    fig = px.scatter(df, x=feature, y="Default_Label", color="Default",
                     title=f"{feature} vs Default Probability",
                     trendline="ols")
    return fig

@app.callback(
    Output("stock-forecast", "figure"),
    Input("feature", "value")
)
def update_forecast_plot(_):
    fig = px.line(forecast_df, x="Date", y="Forecasted_Price", title="Stock Price Forecast (ARIMA)",
                  color_discrete_sequence=["blue"])
    fig.add_scatter(x=forecast_df["Date"], y=forecast_df["LSTM_Forecast"], mode="lines", 
                    name="LSTM Forecast", line=dict(color="red"))
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8060)

C:\Users\Dom3n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\Dom3n\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning:

invalid value encountered in double_scalars

